## Imports

In [0]:
!pip install dnspython

In [0]:
import dns
import pymongo
import pandas as pd

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## MongoDB Connection

In [0]:
# Connect to MongoDB
client = pymongo.MongoClient("mongodb+srv://columbia:<password>@bigdataanalytics-d97sj.gcp.mongodb.net/test?retryWrites=true&w=majority")


## Cleaning & Uploading

In [0]:
# Read and upload data (ONLY PERFORM ONCE)
df = pd.read_csv("/content/drive/My Drive/Big Data Analytics/Data/fires_processed.csv")
# Drop NaN
df.dropna()

# Only keep the required columns
df = df[['MONTH','DAY_OF_WEEK', 'STATE', 'STAT_CAUSE_DESCR', 'FIRE_NAME',
                   'NWCG_REPORTING_UNIT_NAME','FIRE_SIZE', 'LATITUDE', 'LONGITUDE']]
records = df.to_dict('records')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,12,13,14,15,16,17,18,19,20,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
len(records)

1880465

In [0]:
client['Fires'].fires.insert_many(records)

In [0]:
# Pull Dataset
db = client['Fires']
col = db['fires']

In [0]:
# Fixing the data with Unknown Causes
col.update_many( { "STAT_CAUSE_DESCR": {"$nin": ["Arson", "Children", "Lightning", "Debris Burning", "Miscellaneous", "Equipment Use", "Smoking", "Railroad"] } }, { "$set": { "STAT_CAUSE_DESCR": "Unknown" } })

In [0]:
# Fixing the data with Unknown States
col.update_many( {"$expr": { "$gt": [ { "$strLenCP": "$STATE" }, 2 ] }} , { "$set": { "STATE": "Unknown" } })

## PyMongo Testing

**Sample Queries**

The following are sample queries used to test MongoDB before being ported into Flask web application.

In [0]:
myquery = { "STATE": "CA", "STAT_CAUSE_DESCR": "Debris Burning" }
doc = col.find(myquery)

In [0]:
col.distinct("STAT_CAUSE_DESCR")

['Miscellaneous',
 'Lightning',
 'Debris Burning',
 'Unknown',
 'Equipment Use',
 'Arson',
 'Children',
 'Railroad',
 'Smoking']

In [0]:
doc = col.aggregate([
    {"$group" : {"_id":"$STATE", "count":{"$sum":1}}},
    {"$sort": {"count":-1}},
    {"$limit": 10}
])

In [0]:
for x in doc:
  print(x)

{'_id': 'CA', 'count': 189550}
{'_id': 'GA', 'count': 168867}
{'_id': 'TX', 'count': 142021}
{'_id': 'NC', 'count': 111277}
{'_id': 'FL', 'count': 90261}
{'_id': 'SC', 'count': 81315}
{'_id': 'NY', 'count': 80870}
{'_id': 'MS', 'count': 79230}
{'_id': 'AZ', 'count': 71586}
{'_id': 'AL', 'count': 66570}
